<a href="https://colab.research.google.com/github/thenameisAnurag/SCOA/blob/main/SCOA_3___Word_Matching_and_Parameter_Optimization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import string

# Generate a random population
def gen_pop(length, pop_len):
    pop = []
    for _ in range(pop_len):
        pop.append(''.join(random.choices(string.ascii_lowercase, k=length)))
    return pop

# Fitness function: Evaluate how close an individual is to the target word
def fitness(input_word, individual):
    score = 0
    for i in range(len(input_word)):
        if input_word[i] == individual[i]:
            score += 1
    return score / len(input_word)

# Evaluate the fitness of the population
def eval_population(input_word, population):
    fit_pop = {}
    for individual in population:
        fit_pop[individual] = fitness(input_word, individual)
    return fit_pop

# Select a breeding population based on fitness
def breeding_population(fit_pop):
    population = list(fit_pop.keys())
    fitness_values = list(fit_pop.values())
    members = random.choices(population, weights=fitness_values, k=20)
    bpop = {individual: fit_pop[individual] for individual in members}
    return bpop

# Create a new population through crossover
def create_new_population(breeding_population):
    new_population = []
    breeding_members = list(breeding_population.keys())
    for i in range(0, len(breeding_members), 2):
        parent1 = breeding_members[i]
        parent2 = breeding_members[i + 1 if i + 1 < len(breeding_members) else 0]
        child1, child2 = '', ''
        for j in range(len(parent1)):
            if random.random() < 0.5:
                child1 += parent1[j]
                child2 += parent2[j]
            else:
                child1 += parent2[j]
                child2 += parent1[j]
        new_population.append(child1)
        new_population.append(child2)
    return new_population

# Stop condition: Check if fitness threshold is met
def stop_condition(population, threshold=0.6):
    max_fitness = max(list(population.values()))
    return max_fitness >= threshold

# Genetic Algorithm
def genetic_algorithm():
    input_word = input("Enter Word: ").lower()
    population_size = 100
    population = gen_pop(len(input_word), population_size)
    generation = 0

    while True:
        generation += 1
        fit_pop = eval_population(input_word, population)

        # Print all words in the current population
        print(f"\nGeneration {generation}:")
        for word, fitness_value in fit_pop.items():
            print(f"Word: {word}, Fitness: {fitness_value:.2f}")

        # Check if the stop condition is met
        if stop_condition(fit_pop, threshold=1.0):
            members = list(fit_pop.keys())
            fit = list(fit_pop.values())
            print(f"\nWord guessed or threshold met in generation {generation}")
            best_match = members[fit.index(max(fit))]
            print(f"Best matching word: {best_match} with fitness {max(fit):.2f}")
            break

        # Select breeding population and create a new population
        breeding_pop = breeding_population(fit_pop)
        population = create_new_population(breeding_pop)

# Run the Genetic Algorithm
genetic_algorithm()


Enter Word: pune

Generation 1:
Word: ioqv, Fitness: 0.00
Word: ehqh, Fitness: 0.00
Word: llky, Fitness: 0.00
Word: ysqx, Fitness: 0.00
Word: vcaq, Fitness: 0.00
Word: mlmi, Fitness: 0.00
Word: ifxi, Fitness: 0.00
Word: xnqt, Fitness: 0.00
Word: jonb, Fitness: 0.25
Word: wlyp, Fitness: 0.00
Word: qfcn, Fitness: 0.00
Word: qkfq, Fitness: 0.00
Word: olfv, Fitness: 0.00
Word: glxp, Fitness: 0.00
Word: tnml, Fitness: 0.00
Word: dkyx, Fitness: 0.00
Word: smwc, Fitness: 0.00
Word: ksfi, Fitness: 0.00
Word: wdnp, Fitness: 0.25
Word: vbem, Fitness: 0.00
Word: avmx, Fitness: 0.00
Word: wzry, Fitness: 0.00
Word: fkgn, Fitness: 0.00
Word: zyud, Fitness: 0.00
Word: ozzi, Fitness: 0.00
Word: aauo, Fitness: 0.00
Word: fvic, Fitness: 0.00
Word: gwge, Fitness: 0.25
Word: czda, Fitness: 0.00
Word: cuuv, Fitness: 0.25
Word: apfw, Fitness: 0.00
Word: drge, Fitness: 0.25
Word: vijl, Fitness: 0.00
Word: nkvm, Fitness: 0.00
Word: vwyx, Fitness: 0.00
Word: fihr, Fitness: 0.00
Word: vfpa, Fitness: 0.00
Word: 

In [5]:
# Install the DEAP library
!pip install deap

import random
import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from deap import base, creator, tools, algorithms

# Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Define the fitness function
def svm_fitness(individual):
    # Ensure C and gamma are positive
    C = max(individual[0], 0.0001)
    gamma = max(individual[1], 0.0001)

    # Define the SVM model with current hyperparameters
    model = SVC(C=C, gamma=gamma)

    # Perform cross-validation to compute fitness (accuracy)
    accuracy = cross_val_score(model, X, y, cv=5).mean()
    return accuracy,

# Create the fitness function and individual (chromosome) structure
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # We aim to maximize accuracy
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define the parameter space (e.g., C and gamma)
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0.1, 100)   # C: 0.1 to 100
toolbox.register("attr_float2", random.uniform, 0.0001, 1) # gamma: 0.0001 to 1

# Structure of an individual
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_float, toolbox.attr_float2), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic algorithm operations
toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Crossover
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)  # Mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection
toolbox.register("evaluate", svm_fitness)  # Evaluation

# Run the genetic algorithm
def main():
    # Initialize population
    pop = toolbox.population(n=20)  # Population size of 20
    hof = tools.HallOfFame(1)       # Hall of Fame to store the best individual
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", np.max)  # Record the max fitness in each generation
    stats.register("mean", np.mean)  # Record the mean fitness in each generation

    # Run the evolution using the eaSimple algorithm
    algorithms.eaSimple(
        pop,
        toolbox,
        cxpb=0.7,    # Crossover probability
        mutpb=0.2,   # Mutation probability
        ngen=20,     # Number of generations
        stats=stats,
        halloffame=hof,
        verbose=True
    )

    # Print the best individual and its fitness
    print("\nBest individual: ", hof[0])
    print("Best fitness: ", hof[0].fitness.values[0])
    print(f"Optimized Parameters: C={hof[0][0]}, gamma={hof[0][1]}")

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.9 MB/s eta 0:00:00
gen	nevals	max 	mean 
0  	20    	0.98	0.962
1  	12    	0.98	0.971667
2  	14    	0.98	0.969333
3  	16    	0.986667	0.970667
4  	15    	0.986667	0.972667
5  	18    	0.986667	0.975333
6  	17    	0.986667	0.981333
7  	18    	0.986667	0.982   
8  	15    	0.986667	0.982333
9  	14    	0.986667	0.979333
10 	13    	0.986667	0.986   
11 	17    	0.986667	0.980667
12 	16    	0.986667	0.985333
13 	15    	0.986667	0.986667
14 	18    	0.986667	0.979333
15 	20    	0.986667	0.979   
16 	16    	0.986667	0.981667
17 	14    	0.986667	0.979333
18 	14    	0.986667	0.982667
19 	18    	0.986667	0.986667
20 	14    	0.986667	0.986333

Best individual:  [6.747466076758169, 0.11733706338393768]
Best fitness:  0.9866666666666667
Optimized Parameters: C=6.747466076758169, gamma=0.11733706338393768
